# Feature engineering (Members)

This files generates feautures for "members.csv" and exports the file into a folder called data "final_members". In this notebook, more changes to "members" can be made.
In the notebook "algorithm_solution" load "final_members" in.

# 5 Members

## 5.1 Loading the data

In [39]:
#Import the relevant libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mpld3
import seaborn as sns
import matplotlib.dates as mdates
import time
from datetime import datetime
from sklearn import preprocessing

#Configure Panda
pd.options.display.width = 200

In [40]:
#Load members and transactions in
members= pd.read_csv("data/members_v3.csv")
transactions = pd.read_csv("data/transactions_v2.csv")

In [41]:
#Look at the first values in members:
print("members:")
print(members.head())

members:
                                           msno  city  bd  gender  registered_via  registration_init_time
0  Rb9UwLQTrxzBVwCB6+bCcSQWZ9JiNLC9dXtM1oEsZA8=     1   0     NaN              11                20110911
1  +tJonkh+O1CA796Fm5X60UMOtB6POHAwPjbTRVl/EuU=     1   0     NaN               7                20110914
2  cV358ssn7a0f7jZOwGNWS07wCKVqxyiImJUX6xcIwKw=     1   0     NaN              11                20110915
3  9bzDeJP6sQodK73K5CBlJ6fgIQzPeLnRl0p5B77XP+g=     1   0     NaN              11                20110915
4  WFLY3s7z4EZsieHCt63XrsdtfTEmJ+2PnnKLH5GY4Tk=     6  32  female               9                20110915


In [42]:
#Look at the first values in transactions:
print("members:")
print(transactions.head())

members:
                                           msno  payment_method_id  payment_plan_days  plan_list_price  actual_amount_paid  is_auto_renew  transaction_date  membership_expire_date  is_cancel
0  ++6eU4LsQ3UQ20ILS7d99XK8WbiVgbyYL4FUgzZR134=                 32                 90              298                 298              0          20170131                20170504          0
1  ++lvGPJOinuin/8esghpnqdljm6NXS8m8Zwchc7gOeA=                 41                 30              149                 149              1          20150809                20190412          0
2  +/GXNtXWQVfKrEDqYAzcSw2xSPYMKWNj22m+5XkVQZc=                 36                 30              180                 180              1          20170303                20170422          0
3  +/w1UrZwyka4C9oNH3+Q8fUf3fD8R3EwWrx57ODIsqk=                 36                 30              180                 180              1          20170329                20170331          1
4  +00PGzKTYqtnb65mPKPyeHXcZEwqiEzkt

## 5.2 Features

### 5.2.1 Hot-encoding of city

In [43]:
#One-hot encode the cities. 
#Instead of having a variable called city with values from 1-22, the alorithm performs better with 0's and 1's ->onehot encoding

#Generate our final file called final1
final1 = members

#One-hot encode city and save it into city_encde
city_encode = pd.get_dummies(final1['city'],prefix='city')

#Drop variable city in final2, as it is no longer needed
final1=final1.drop('city',axis=1)

#Join the encoded city_encode
final1 = final1.join(city_encode)

final1.head()


,msno,bd,gender,registered_via,registration_init_time,city_1,city_3,city_4,city_5,city_6,...,city_13,city_14,city_15,city_16,city_17,city_18,city_19,city_20,city_21,city_22
0,Rb9UwLQTrxzBVwCB6+bCcSQWZ9JiNLC9dXtM1oEsZA8=,0,NaN,11,20110911,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,+tJonkh+O1CA796Fm5X60UMOtB6POHAwPjbTRVl/EuU=,0,NaN,7,20110914,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,cV358ssn7a0f7jZOwGNWS07wCKVqxyiImJUX6xcIwKw=,0,NaN,11,20110915,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9bzDeJP6sQodK73K5CBlJ6fgIQzPeLnRl0p5B77XP+g=,0,NaN,11,20110915,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,WFLY3s7z4EZsieHCt63XrsdtfTEmJ+2PnnKLH5GY4Tk=,32,female,9,20110915,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


### 5.2.2 Hot-encoding of the year

In [44]:
#transform integer dates to data type: date
final1['registration_init_time'] = final1.registration_init_time.apply(lambda x: datetime.strptime(str(int(x)), "%Y%m%d").date() if pd.notnull(x) else "NAN" )


IGNORE THIS BOX FOR NOW


#Give the year and month of registration in two new columns
date = pd.DataFrame(columns=['reg_year_2017'])

date.reg_year_2017=pd.DatetimeIndex(final1['registration_init_time']).year


mask=(date.reg_year_2017==2017)
mask_not=(date.reg_year_2017<2017)
column_name='reg_year_2017'
date.loc[mask,column_name]=1
date.loc[mask_not,column_name]=0



#Drop variable registration_init_time in final2, as it is no longer needed
final1=final1.drop('registration_init_time',axis=1)
#Join the two new columns
final1=final1.join(date)

print(final1)


In [45]:
#Create a dataframe containing the colum reg_year
date = pd.DataFrame(columns=['reg_year'])
#save the year of registration in it
date.reg_year=pd.DatetimeIndex(final1['registration_init_time']).year

#Drop variable registration_init_time in final2, as it is no longer needed
final1=final1.drop('registration_init_time',axis=1)
#Join the two dataframes
final1=final1.join(date)

#get one-hot encoding columns
year_encode = pd.get_dummies(final1['reg_year'],prefix='reg_year')
#Drop not so relevant years, see data exploraion file for members
year_encode=year_encode.drop(['reg_year_2004','reg_year_2005','reg_year_2006','reg_year_2007','reg_year_2008','reg_year_2009','reg_year_2010','reg_year_2011'],axis=1)

#Join and drop
final1 = final1.join(year_encode)
final1=final1.drop('reg_year',axis=1)

final1.head()


,msno,bd,gender,registered_via,city_1,city_3,city_4,city_5,city_6,city_7,...,city_19,city_20,city_21,city_22,reg_year_2012,reg_year_2013,reg_year_2014,reg_year_2015,reg_year_2016,reg_year_2017
0,Rb9UwLQTrxzBVwCB6+bCcSQWZ9JiNLC9dXtM1oEsZA8=,0,NaN,11,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,+tJonkh+O1CA796Fm5X60UMOtB6POHAwPjbTRVl/EuU=,0,NaN,7,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,cV358ssn7a0f7jZOwGNWS07wCKVqxyiImJUX6xcIwKw=,0,NaN,11,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9bzDeJP6sQodK73K5CBlJ6fgIQzPeLnRl0p5B77XP+g=,0,NaN,11,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,WFLY3s7z4EZsieHCt63XrsdtfTEmJ+2PnnKLH5GY4Tk=,32,female,9,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


### 5.2.3 One-hot encoding of registration method

In [46]:
#get the dummy columns
reg_encode = pd.get_dummies(final1['registered_via'],prefix='reg')

#drop the not so relevant columns (see data exploration file for members)
reg_encode=reg_encode.drop(['reg_-1','reg_1','reg_2','reg_5','reg_6','reg_8','reg_10','reg_13','reg_14','reg_16','reg_17','reg_18','reg_19'],axis=1)

#Join and drop
final1 = final1.join(reg_encode)
final1=final1.drop('registered_via',axis=1)

final1.head()


,msno,bd,gender,city_1,city_3,city_4,city_5,city_6,city_7,city_8,...,reg_year_2013,reg_year_2014,reg_year_2015,reg_year_2016,reg_year_2017,reg_3,reg_4,reg_7,reg_9,reg_11
0,Rb9UwLQTrxzBVwCB6+bCcSQWZ9JiNLC9dXtM1oEsZA8=,0,NaN,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,+tJonkh+O1CA796Fm5X60UMOtB6POHAwPjbTRVl/EuU=,0,NaN,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,cV358ssn7a0f7jZOwGNWS07wCKVqxyiImJUX6xcIwKw=,0,NaN,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,9bzDeJP6sQodK73K5CBlJ6fgIQzPeLnRl0p5B77XP+g=,0,NaN,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,WFLY3s7z4EZsieHCt63XrsdtfTEmJ+2PnnKLH5GY4Tk=,32,female,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0


In [47]:
#print which columns are left for registration metod 
reg_encode.head()
final1.head()

,msno,bd,gender,city_1,city_3,city_4,city_5,city_6,city_7,city_8,...,reg_year_2013,reg_year_2014,reg_year_2015,reg_year_2016,reg_year_2017,reg_3,reg_4,reg_7,reg_9,reg_11
0,Rb9UwLQTrxzBVwCB6+bCcSQWZ9JiNLC9dXtM1oEsZA8=,0,NaN,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,+tJonkh+O1CA796Fm5X60UMOtB6POHAwPjbTRVl/EuU=,0,NaN,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,cV358ssn7a0f7jZOwGNWS07wCKVqxyiImJUX6xcIwKw=,0,NaN,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,9bzDeJP6sQodK73K5CBlJ6fgIQzPeLnRl0p5B77XP+g=,0,NaN,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,WFLY3s7z4EZsieHCt63XrsdtfTEmJ+2PnnKLH5GY4Tk=,32,female,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0


### 5.2.4 Young people

In [48]:
#We create a column with young people under 24, because they are more likely to churn
#save bd in a vector for new feature later on
bd=final1.bd

#create indexes
mask=(final1.bd<24)
mask_not=(final1.bd<1)
mask_not2=(final1.bd>23)

column_name='bd'
#change birthday indexes
final1.loc[mask,column_name]=1
final1.loc[mask_not,column_name]=0
final1.loc[mask_not2,column_name]=0

final1.head()


,msno,bd,gender,city_1,city_3,city_4,city_5,city_6,city_7,city_8,...,reg_year_2013,reg_year_2014,reg_year_2015,reg_year_2016,reg_year_2017,reg_3,reg_4,reg_7,reg_9,reg_11
0,Rb9UwLQTrxzBVwCB6+bCcSQWZ9JiNLC9dXtM1oEsZA8=,0,NaN,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,+tJonkh+O1CA796Fm5X60UMOtB6POHAwPjbTRVl/EuU=,0,NaN,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,cV358ssn7a0f7jZOwGNWS07wCKVqxyiImJUX6xcIwKw=,0,NaN,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,9bzDeJP6sQodK73K5CBlJ6fgIQzPeLnRl0p5B77XP+g=,0,NaN,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,WFLY3s7z4EZsieHCt63XrsdtfTEmJ+2PnnKLH5GY4Tk=,0,female,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0


### 5.2.5 Normalization of birthday

In [49]:
bd_df = pd.DataFrame(columns=['bd_norm'])
bd_df.bd_norm=bd

#set outliers to mean:
mask=(bd_df.bd_norm<0)
mask2=(bd_df.bd_norm>90)
column_name='bd_norm'
bd_df.loc[mask,column_name]=24
bd_df.loc[mask_not,column_name]=24

#normalize
bd_df.bd_norm=bd_df.bd_norm/90

final1 = final1.join(bd_df)


In [50]:
print(bd_df.bd_norm)

0          0.266667
1          0.266667
2          0.266667
3          0.266667
4          0.000000
5          0.000000
6          0.266667
7          0.000000
8          0.011111
9          0.000000
10         0.266667
11         0.011111
12         0.000000
13         0.000000
14         0.000000
15         0.266667
16         0.266667
17         0.000000
18         0.266667
19         0.011111
20         0.011111
21         0.011111
22         0.011111
23         0.000000
24         0.011111
25         0.011111
26         0.000000
27         0.000000
28         0.000000
29         0.266667
             ...   
6769443    0.266667
6769444    0.266667
6769445    0.266667
6769446    0.266667
6769447    0.266667
6769448    0.011111
6769449    0.266667
6769450    0.266667
6769451    0.266667
6769452    0.011111
6769453    0.266667
6769454    0.000000
6769455    0.000000
6769456    0.266667
6769457    0.266667
6769458    0.266667
6769459    0.266667
6769460    0.266667
6769461    0.266667


## 5.4 Check and export file

In [51]:
#dropping bd gender, regstered_via and regisration_init_time to test if city encoding helps algorithm performing
final1=final1.drop(['gender'],axis=1)
final1.head()

,msno,bd,city_1,city_3,city_4,city_5,city_6,city_7,city_8,city_9,...,reg_year_2014,reg_year_2015,reg_year_2016,reg_year_2017,reg_3,reg_4,reg_7,reg_9,reg_11,bd_norm
0,Rb9UwLQTrxzBVwCB6+bCcSQWZ9JiNLC9dXtM1oEsZA8=,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0.266667
1,+tJonkh+O1CA796Fm5X60UMOtB6POHAwPjbTRVl/EuU=,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0.266667
2,cV358ssn7a0f7jZOwGNWS07wCKVqxyiImJUX6xcIwKw=,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0.266667
3,9bzDeJP6sQodK73K5CBlJ6fgIQzPeLnRl0p5B77XP+g=,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0.266667
4,WFLY3s7z4EZsieHCt63XrsdtfTEmJ+2PnnKLH5GY4Tk=,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0.000000


In [52]:
#Check for null-values:
final1.isnull().sum()

msno             0
bd               0
city_1           0
city_3           0
city_4           0
city_5           0
city_6           0
city_7           0
city_8           0
city_9           0
city_10          0
city_11          0
city_12          0
city_13          0
city_14          0
city_15          0
city_16          0
city_17          0
city_18          0
city_19          0
city_20          0
city_21          0
city_22          0
reg_year_2012    0
reg_year_2013    0
reg_year_2014    0
reg_year_2015    0
reg_year_2016    0
reg_year_2017    0
reg_3            0
reg_4            0
reg_7            0
reg_9            0
reg_11           0
bd_norm          0
dtype: int64

In [53]:
#export file, can take up to 1 min
final1.to_csv('data/final_members.csv', index=False)
print("Done!")


Done!
